<a href="https://colab.research.google.com/github/tashfeen786/Decision_Tree_from_scratch_-without_using_libraries_like_sklearn/blob/main/Decision_Tree_from_scratch_(without_using_libraries_like_sklearn).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [8]:
# Sample dataset
dataset = [
    ['Sunny', 'Hot', 'High', 'Weak', 'No'],
    ['Sunny', 'Hot', 'High', 'Strong', 'No'],
    ['Overcast', 'Hot', 'High', 'Weak', 'Yes'],
    ['Rain', 'Mild', 'High', 'Weak', 'Yes'],
    ['Rain', 'Cool', 'Normal', 'Weak', 'Yes'],
    ['Rain', 'Cool', 'Normal', 'Strong', 'No'],
    ['Overcast', 'Cool', 'Normal', 'Strong', 'Yes'],
    ['Sunny', 'Mild', 'High', 'Weak', 'No'],
    ['Sunny', 'Cool', 'Normal', 'Weak', 'Yes'],
    ['Rain', 'Mild', 'Normal', 'Weak', 'Yes'],
    ['Sunny', 'Mild', 'Normal', 'Strong', 'Yes'],
    ['Overcast', 'Mild', 'High', 'Strong', 'Yes'],
    ['Overcast', 'Hot', 'Normal', 'Weak', 'Yes'],
    ['Rain', 'Mild', 'High', 'Strong', 'No']
]

features = ['Outlook', 'Temperature', 'Humidity', 'Wind']


In [9]:
import math

def entropy(data):
    labels = [row[-1] for row in data]
    label_counts = {}
    for label in labels:
        label_counts[label] = label_counts.get(label, 0) + 1
    total = len(labels)
    return -sum((count / total) * math.log2(count / total) for count in label_counts.values())


In [10]:
def info_gain(data, feature_index):
    total_entropy = entropy(data)
    values = set(row[feature_index] for row in data)
    subset_entropy = 0.0
    for value in values:
        subset = [row for row in data if row[feature_index] == value]
        subset_entropy += len(subset) / len(data) * entropy(subset)
    return total_entropy - subset_entropy


In [4]:
def build_tree(data, features):
    labels = [row[-1] for row in data]

    # If only one class remains
    if labels.count(labels[0]) == len(labels):
        return labels[0]

    # If no features left
    if len(features) == 0:
        return max(set(labels), key=labels.count)

    # Choose best feature
    gains = [info_gain(data, i) for i in range(len(features))]
    best_index = gains.index(max(gains))
    best_feature = features[best_index]

    tree = {best_feature: {}}
    values = set(row[best_index] for row in data)

    for value in values:
        subset = [row[:best_index] + row[best_index+1:] for row in data if row[best_index] == value]
        sub_features = features[:best_index] + features[best_index+1:]
        subtree = build_tree(subset, sub_features)
        tree[best_feature][value] = subtree

    return tree


In [5]:
def predict(tree, features, instance):
    if not isinstance(tree, dict):
        return tree
    root = next(iter(tree))
    index = features.index(root)
    value = instance[index]
    subtree = tree[root].get(value)
    if subtree is None:
        return "Unknown"
    return predict(subtree, features[:index] + features[index+1:], instance[:index] + instance[index+1:])


In [6]:
# Build the decision tree
tree = build_tree(dataset, features)
print("Decision Tree:")
print(tree)

# Predict example
test_instance = ['Sunny', 'Cool', 'High', 'Strong']
result = predict(tree, features, test_instance)
print(f"Prediction for {test_instance}: {result}")


Decision Tree:
{'Outlook': {'Sunny': {'Humidity': {'High': 'No', 'Normal': 'Yes'}}, 'Overcast': 'Yes', 'Rain': {'Wind': {'Strong': 'No', 'Weak': 'Yes'}}}}
Prediction for ['Sunny', 'Cool', 'High', 'Strong']: No


In [11]:
def print_tree(tree, indent=""):
    if not isinstance(tree, dict):
        print(indent + "Predict:", tree)
        return
    for key, value in tree.items():
        for val, subtree in value.items():
            print(indent + f"{key} = {val}")
            print_tree(subtree, indent + "   ")


In [12]:
print("Decision Tree:")
print_tree(tree)


Decision Tree:
Outlook = Sunny
   Humidity = High
      Predict: No
   Humidity = Normal
      Predict: Yes
Outlook = Overcast
   Predict: Yes
Outlook = Rain
   Wind = Strong
      Predict: No
   Wind = Weak
      Predict: Yes


In [14]:
from graphviz import Digraph
import uuid

def build_graph(tree, dot=None, parent=None, edge_label=""):
    if dot is None:
        dot = Digraph()

    if not isinstance(tree, dict):
        node_id = str(uuid.uuid4())
        dot.node(node_id, f"Predict: {tree}", shape='box', style='filled', color='lightblue')
        if parent:
            dot.edge(parent, node_id, label=edge_label)
        return dot

    root = next(iter(tree))
    node_id = str(uuid.uuid4())
    dot.node(node_id, root, shape='ellipse', style='filled', color='lightgreen')

    if parent:
        dot.edge(parent, node_id, label=edge_label)

    for value, subtree in tree[root].items():
        build_graph(subtree, dot, node_id, str(value))

    return dot


In [15]:
graph = build_graph(tree)
graph.render('decision_tree', view=True, format='png')  # saves and opens the tree as image


'decision_tree.png'